# Visualization

> Visualization tools such as piano keys and guitar fretboards.

In [ ]:
#|default_exp viz

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
from fasthtml import *
from fastcore.all import *
from fasthtml.jupyter import render_ft
from IPython.display import HTML

from musy import Note, Chord, Scale

render_ft()

# Base

Here we will create an Instrument base class to for code reusability.

In [ ]:
#|export
class Instrument:
    def __ft__(self, midi_notes: list[int] = []):
        """Base class for instrument visualizations. Override this method in subclasses."""
        raise NotImplementedError("Subclasses must implement __ft__")
    
    def visualize_note(self, note: Note):
        """Visualize a single note on the instrument."""
        return self.__ft__([note.midi])
    
    def visualize_notes(self, notes: list[Note]):
        """Visualize multiple notes on the instrument."""
        return self.__ft__([n.midi for n in notes])
    
    def visualize_chord(self, chord: Chord):
        """Visualize a chord on the instrument."""
        return self.__ft__([n.midi for n in chord.notes])
    
    def visualize_scale(self, scale: Scale, root: str = "C", octs = 2):
        """Visualize a scale on the instrument across multiple octaves."""
        return self.__ft__([n.midi for oct in range(1, octs+1) for n in scale.get_notes(root, oct=oct)])
    
    def __call__(self, highlight = None):
        """Main entry point for visualization. Handles different input types."""
        if not highlight:
            return self.__ft__([])
        elif isinstance(highlight, Note):
            return self.visualize_note(highlight)
        elif isinstance(highlight, Chord):
            return self.visualize_chord(highlight)
        elif isinstance(highlight, Scale):
            return self.visualize_scale(highlight)
        elif isinstance(highlight, list):
            return self.__ft__([n.midi for n in highlight])
        else:
            raise ValueError(f"Unsupported type: {type(highlight)}")

# Piano

The `Piano` object is the basic piano visualization on which we can place `Note`, `Interval`, `Chord` and `Scale` objects.

In [ ]:
#|export
class Piano(Instrument):
    def __ft__(self, midi_notes: list[int] = []):
        midi_range = range(min(midi_notes or [60]), max(midi_notes or [60]) + 1)
        octaves = range(min(midi_range) // 12, max(midi_range) // 12 + 1)
        white_keys = [(note, Note(note, oct).midi) for oct in octaves for note in ['C', 'D', 'E', 'F', 'G', 'A', 'B']]
        black_keys = [(sharp, Note(sharp, oct).midi, i + (oct - min(octaves)) * 7) 
                     for oct in octaves for i, sharp in enumerate(['C#', 'D#', 'F#', 'G#', 'A#'])]
        
        css = Style("""
.piano { background: #222; padding: 20px 0; position: relative; }
.white-keys { display: flex; }
.white-key, .black-key { text-align: center; font-family: Arial; position: relative; }
.white-key { width: 40px; height: 125px; background: #fff; border: 1px solid #000; 
    color: #111; font-size: 18px; line-height: 200px; z-index: 1; }
.black-key { width: 20px; height: 80px; background: #000; color: #fff; border: 1px solid #333;
    position: absolute; z-index: 2; line-height: 100px; font-size: 14px; top: 20px; }
.highlight { background: #ff0 !important; color: #000 !important; }
""")
        white_divs = [Div(note, cls=f"white-key{' highlight' if midi in midi_notes else ''}") for note, midi in white_keys]
        black_divs = [Div(note, cls=f"black-key{' highlight' if midi in midi_notes else ''}", style=f"left:{(idx + 1) * 40 - 11}px") 
                     for note, midi, idx in black_keys]
        return HTML(css + Div(Div(*white_divs, cls="white-keys"), *black_divs, cls="piano", style=f"width:{len(white_keys)*40}px"))

## Initialization

To get an empty piano, just call on an initialized `Piano` object without arguments.

In [ ]:
piano = Piano()
piano()

## Highlighting

We can highlight single `Note` objects with the `visualize_note` method.

For example, here we highlight the `C#` notes on the piano.

In [ ]:
piano.visualize_note(Note("C#"))

`Chord` objects can be visualized on the piano by passing a chord to `visualize_chord`.

In [ ]:
# Notes for a Cmaj7 chord
chord = Chord.from_short("Cmaj7")
chord.notes

[C4, E4, G4, B4]

In [ ]:
piano.visualize_chord(chord)

A scale can be highlighted by calling `visualize_scale` with a `Scale` object and a root note.

In [ ]:
major = Scale("major")
piano.visualize_scale(major, root="D")

You can decide to visualize more octaves with the `octs` parameter.

In [ ]:
double_harmonic_major = Scale("double harmonic major")
piano.visualize_scale(double_harmonic_major, root="C", octs=3)

# Guitar

Just like with `Piano` we can visualize `Note`, `Chord` and `Scale` objects on a guitar fretboard.

In [ ]:
#|export
class Guitar(Instrument):
    def __ft__(self, midi_notes: list[int] = []):
        num_frets = 22
        string_names = ['E', 'B', 'G', 'D', 'A', 'E']
        standard_tuning = [64, 59, 55, 50, 45, 40]

        # Fret numbers row
        fret_nums = [Div(str(fret), cls="guitar-fret-num") for fret in range(num_frets+1)]
        fret_nums = Div(Div("", cls="guitar-string-name"), *fret_nums, cls="guitar-fret-nums")

        # Fretboard grid
        rows = []
        for name, open_midi in zip(string_names, standard_tuning):
            cells = []
            for fret in range(num_frets+1):
                midi = open_midi + fret
                if midi in midi_notes:
                    cells.append(Div(str(Note.from_midi(midi)), cls="guitar-note"))
                else:
                    cells.append(Div("", cls="guitar-cell"))
            rows.append(Div(Div(name, cls="guitar-string-name"), *cells, cls="guitar-row"))

        # CSS
        css = Style("""
.guitar-fretboard { background: #c49e60; border-radius: 8px; padding: 8px; display: inline-block; }
.guitar-fret-nums { display: flex; font-size: 12px; color: #333; margin-bottom: 2px; }
.guitar-fret-num { width: 32px; text-align: center; font-weight: bold; border-right: 2.5px solid #888; }
.guitar-dot { height: 6px; text-align: center; color: #444; font-size: 10px; }
.guitar-row { display: flex; align-items: center; position: relative; }
.guitar-string-name { width: 24px; text-align: right; margin-right: 0px; font-weight: bold; color: #444; }
.guitar-cell, .guitar-note { width: 32px; height: 32px; display: flex; align-items: center; justify-content: center; border-right: 2.5px solid #888; position: relative; }
.guitar-cell { border-bottom: 1.2px solid #bbb; background: none; }
.guitar-note { background: rgba(255, 255, 0, 0.7); color: #222; 
""")
        board = Div(
            fret_nums,
            *rows,
            cls="guitar-fretboard"
        )
        return HTML(css + board)
    
    def visualize_chord(self, chord: Chord):
        return self.__ft__([Note(str(n), oct).midi for oct in range(1, 8) for n in list(chord)])
    
    def visualize_scale(self, scale: Scale, root: str = "C"):
        return self.__ft__([n.midi for oct in range(1, 8) for n in scale.get_notes(root, oct=oct)])

In [ ]:
guitar = Guitar()
guitar.visualize_note(Note("C", oct=5))

In [ ]:
guitar.visualize_chord(Chord.from_short("Cmaj7"))

In [ ]:
guitar.visualize_scale(Scale("major"), root="C")

----------------

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()